In [11]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('data/TSLA.csv')

# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set date as index
df.set_index('Date', inplace=True)

# Calculate the end of week dates
df['Week'] = df.index.to_series().dt.isocalendar().week
df['Year'] = df.index.to_series().dt.isocalendar().year
df['TradingWeek'] = (
    df['Year'].astype(str) + '-' 
    + df['Week'].astype(str).apply(lambda x: str(x).zfill(2))
    )

# Filter out the end trading day of each week
weekly_df = df.groupby('TradingWeek').last()

# Calculate the last Friday's close price
weekly_df['PrevClose'] = weekly_df['Close'].shift(1)

# Initialize the profit column
weekly_df['Profit'] = 0.0

# Calculate profit or loss for each week
for i in range(1, len(weekly_df)):
    prev_close = weekly_df.iloc[i-1]['Close']
    this_close = weekly_df.iloc[i]['Close']
    premium = 0.01
    strike_factor = 1.15
    if this_close <= prev_close * strike_factor:
        profit = prev_close * premium
    else:
        profit = prev_close * premium - (this_close - prev_close * strike_factor)
    weekly_df.iloc[i, weekly_df.columns.get_loc('Profit')] = profit

# Calculate the cumulative profit
weekly_df['CumulativeProfit'] = weekly_df['Profit'].cumsum()

# Select only necessary columns
output_df = weekly_df[['PrevClose', 'Close', 'Profit', 'CumulativeProfit']].reset_index()
recent_years_df = output_df[output_df['TradingWeek'] >= '2020-1']
recent_years_df

,TradingWeek,PrevClose,Close,Profit,CumulativeProfit
505,2020-10,44.532665,46.898666,0.445327,57.680692
506,2020-11,46.898666,36.441334,0.468987,58.149679
507,2020-12,36.441334,28.502001,0.364413,58.514092
508,2020-13,28.502001,34.290668,-1.228347,57.285746
509,2020-14,34.290668,32.000668,0.342907,57.628652
...,...,...,...,...,...
675,2023-23,213.970001,244.399994,2.139700,286.921364
676,2023-24,244.399994,260.540009,2.444000,289.365364
677,2023-25,260.540009,256.600006,2.605400,291.970764
678,2023-26,256.600006,261.769989,2.566000,294.536764
